<a href="https://colab.research.google.com/github/Juethro/DataQuest_2023_NewQuest/blob/kevin/DataQuest2023_revisi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Load Library

In [ ]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c tahap-penyisihan-oq-dataquestua

In [ ]:
!unzip tahap-penyisihan-oq-dataquestua.zip -d datasets

Archive:  tahap-penyisihan-oq-dataquestua.zip
replace datasets/sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 
error:  invalid response [{ENTER}]
replace datasets/sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 
error:  invalid response [{ENTER}]
replace datasets/sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
dft = pd.read_csv("datasets/train.csv")
dft

#Statistika Deskriptif

In [ ]:
dft.info()

In [ ]:
len(dft)

In [ ]:
dft.describe().transpose()

#Data Prepocessing

In [ ]:
dft.drop("time-zone", axis=1, inplace=True)
dft['temp'] = dft['temp'].str.extract(r'(\d+\.\d+|\d+)').astype(float)
dft['d_point'] = dft['d_point'].str.extract(r'(\d+\.\d+|\d+)').astype(float)
dft['feels'] = dft['feels'].str.extract(r'(\d+\.\d+|\d+)').astype(float)
dft['min_temp'] = dft['min_temp'].str.extract(r'(\d+\.\d+|\d+)').astype(float)
dft['max_temp'] = dft['max_temp'].str.extract(r'(\d+\.\d+|\d+)').astype(float)
dft['prssr'] = dft['prssr'].str.extract(r'(\d+\.\d+|\d+)').astype(float)
dft['hum'] = dft['hum'].str.extract(r'(\d+\.\d+|\d+)').astype(float)
dft['wind_spd'] = dft['wind_spd'].str.extract(r'(\d+\.\d+|\d+)').astype(float)
dft['wind_deg'] = dft['wind_deg'].str.extract(r'(\d+\.\d+|\d+)').astype(float)
dft["clouds"] = dft["clouds"].str.extract(r'(\d+\.\d+|\d+)').astype(float)

dft["rain_1h"] = dft['rain_1h'].str.replace('zero', '0')
dft['rain_1h'] = dft['rain_1h'].replace('-1m', '-1')
dft["rain_1h"] = dft['rain_1h'].str.extract(r'(\d+\.\d+|\d+)').astype(float)

dft["rain_3h"] = 0
dft["snow_1h"] = 0
dft["snow_3h"] = 0

In [ ]:
dft.info()

In [ ]:
dft.head(20)

In [ ]:
dft.info()

In [ ]:
dft.isna().sum()

In [ ]:
dft.rain_1h

In [ ]:
dft['rain_1h'].fillna(0, inplace=True)

#Menghapus Variabel

In [ ]:
columns_to_drop = ['datetime', 'visibility', 'sea_level', 'grnd_level']

dft = dft.drop(columns_to_drop, axis =1)

# # Menghapus dan mengambil kolom-kolom tersebut
# for column in columns_to_drop:
#     removed_column = dft.pop(column)

In [ ]:
dft.info()

In [ ]:
dft.head()

#Mendeteksi Outlier

In [ ]:
from scipy import stats

columns_to_check = ['temp', 'd_point', 'feels', 'min_temp', 'max_temp', 'prssr', 'hum', 'wind_spd', 'wind_deg', 'rain_1h', 'rain_3h', 'snow_1h', 'snow_3h', 'clouds']

# Hitung Z-score untuk setiap kolom
z_scores = stats.zscore(dft[columns_to_check])

# Tentukan ambang batas Z-score untuk deteksi outlier (misalnya, 2 atau 3)
threshold = 3

# Deteksi outlier
outliers = (z_scores > threshold).any(axis=1)

# Tampilkan baris-baris yang berisi outlier
dft[outliers]

In [ ]:
from scipy import stats

# Tentukan kolom yang akan dianalisis (kolom numerik)
columns_to_check = ['temp', 'd_point', 'feels', 'min_temp', 'max_temp', 'prssr', 'hum', 'wind_spd', 'wind_deg', 'rain_1h', 'rain_3h', 'snow_1h', 'snow_3h', 'clouds']

# Hitung Z-score untuk setiap kolom
z_scores = stats.zscore(dft[columns_to_check])

# Tentukan ambang batas Z-score untuk deteksi outlier (misalnya, 2 atau 3)
threshold = 3

# Deteksi outlier
outliers = (z_scores > threshold).any(axis=1)

# Gantikan outlier dengan nilai rata-rata
for column in columns_to_check:
    mean_value = dft[column].mean()
    dft.loc[outliers, column] = mean_value

# Tampilkan DataFrame yang telah menggantikan outlier dengan nilai rata-rata
dft

#Grafik

In [ ]:
dft.hist(bins=15, figsize=(12,10), alpha=0.6, label=False)

In [ ]:
dft.isna().sum()

#Modelling

##Regresi Linier

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Memisahkan variabel independen (fitur) dan variabel dependen (target)
X = dft[['temp','d_point', 'feels', 'min_temp', 'max_temp', 'prssr', 'hum', 'wind_spd', 'wind_deg', 'rain_3h', 'snow_1h', 'snow_3h', 'clouds']]
y = dft['rain_1h']

# Membagi data menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inisialisasi model regresi linier
model_lr = LinearRegression()

# Melatih model pada data latih
model_lr.fit(X_train, y_train)

# Memprediksi data uji
y_pred = model_lr.predict(X_test)

# Menghitung metrik evaluasi (misalnya, MSE dan R-squared)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Menghitung RMSE
rmse = np.sqrt(mse)

print("Mean Squared Error:", mse)
print("Root Mean Squared Error:", rmse)
print("R-squared:", r2)


##Neural Networks

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Memisahkan variabel independen (fitur) dan variabel dependen (target)
X = dft[['temp', 'd_point', 'feels', 'min_temp', 'max_temp', 'prssr', 'hum', 'wind_spd', 'wind_deg', 'rain_3h', 'snow_1h', 'snow_3h', 'clouds']]
y = dft['rain_1h']

# Membagi data menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inisialisasi model jaringan saraf tiruan
model_nn = Sequential()
model_nn.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
model_nn.add(Dense(64, activation='relu'))
model_nn.add(Dense(1, activation='linear'))

# Kompilasi model
model_nn.compile(loss='mean_squared_error', optimizer='adam')

# Melatih model pada data latih
model_nn.fit(X_train, y_train, epochs=100, batch_size=32)

# Memprediksi data uji
y_pred = model_nn.predict(X_test)

# Menghitung metrik evaluasi (misalnya, MSE dan R-squared)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Menghitung RMSE
rmse = np.sqrt(mse)

print("Mean Squared Error:", mse)
print("Root Mean Squared Error:", rmse)
print("R-squared:", r2)


##XGB

In [ ]:
dft['datetime_iso'] = pd.to_datetime(dft['datetime_iso'])

# Ekstrak fitur-fitur dari kolom datetime_iso
dft['year'] = dft['datetime_iso'].dt.year
dft['month'] = dft['datetime_iso'].dt.month
dft['day'] = dft['datetime_iso'].dt.day
dft['hour'] = dft['datetime_iso'].dt.hour

In [ ]:
dft['visibility'] = pd.to_numeric(dft['visibility'], errors='coerce')  # Mengkonversi ke float
dft['visibility'].fillna(0, inplace=True)  # Mengganti nilai yang hilang dengan 0

In [ ]:
dft['sea_level'] = pd.to_numeric(dft['sea_level'], errors='coerce')

In [ ]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Memisahkan fitur (features) dan target (target)
X = dft.drop(columns=['temp', 'd_point', 'feels', 'min_temp', 'max_temp', 'prssr', 'hum', 'wind_spd', 'wind_deg', 'rain_3h', 'snow_1h', 'snow_3h', 'clouds'])  # Ganti 'target' dengan nama kolom target yang sesuai
y = dft['rain_1h']

# Memisahkan data menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Membuat model XGBoost
model_xgb = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

# Melatih model pada data latih
model_xgb.fit(X_train, y_train)

# Membuat prediksi pada data uji
y_pred = model_xgb.predict(X_test)

# Menghitung mean squared error (MSE) sebagai metrik evaluasi
mse = mean_squared_error(y_test, y_pred)

# Menghitung Root Mean Squared Error (RMSE)
rmse = mse ** 0.5

print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")

##Coba-coba

In [ ]:
#Library for modelling
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

# Pisahkan fitur (variabel independen) dan target (rain_1h) dari dataset
X = dft.drop(columns=['rain_1h', 'datetime_iso'])
y = dft['rain_1h']

In [ ]:
# Bagi data menjadi data pelatihan (train set) dan data pengujian (test set)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
categorical_cols = ['temp', 'd_point', 'feels']  # Ganti dengan kolom-kolom yang sesuai
dft[categorical_cols] = dft[categorical_cols].astype('category')
dtrain = xgb.DMatrix(X_train, label=y_train, enable_categorical=True)

In [ ]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

# Buat dan latih model XGBoost
xgb_model = XGBRegressor(objective='reg:squarederror', random_state=42)
xgb_model.fit(X_train, y_train)

# Membuat prediksi pada data uji
y_pred_xgb = xgb_model.predict(X_test)

# Menghitung Root Mean Squared Error (RMSE)
rmse_xgb = mean_squared_error(y_test, y_pred_xgb, squared=False)
print(f'XGBoost- RMSE: {rmse_xgb}')

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

# Kolom-kolom kategorikal yang ingin diubah
categorical_cols = ['temp', 'd_point', 'feels']  # Ganti dengan kolom-kolom yang sesuai

# Melakukan label encoding untuk setiap kolom kategorikal
label_encoders = {}
for col in categorical_cols:
    label_encoder = LabelEncoder()
    dft[col] = label_encoder.fit_transform(dft[col])
    label_encoders[col] = label_encoder

# Memisahkan fitur (features) dan target (target)
X = dft.drop(columns=['rain_1h', 'datetime_iso'])
y = dft['rain_1h']

# Melakukan pemisahan data menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Membuat dan melatih model XGBoost
xgb_models = XGBRegressor(objective='reg:squarederror', random_state=42)
xgb_models.fit(X_train, y_train)

# Membuat prediksi pada data uji
y_pred_xgb = xgb_models.predict(X_test)

# Menghitung Root Mean Squared Error (RMSE)
rmse_xgb = mean_squared_error(y_test, y_pred_xgb, squared=False)
print(f'XGBoost- RMSE: {rmse_xgb}')

# Uploader

In [ ]:
dft_test = pd.read_csv('datasets/test.csv')

In [ ]:
dft_test

In [ ]:
dft_test['temp'] = dft_test['temp'].str.extract(r'(\d+\.\d+|\d+)').astype(float)
dft_test['d_point'] = dft_test['d_point'].str.extract(r'(\d+\.\d+|\d+)').astype(float)
dft_test['feels'] = dft_test['feels'].str.extract(r'(\d+\.\d+|\d+)').astype(float)
dft_test['min_temp'] = dft_test['min_temp'].str.extract(r'(\d+\.\d+|\d+)').astype(float)
dft_test['max_temp'] = dft_test['max_temp'].str.extract(r'(\d+\.\d+|\d+)').astype(float)
dft_test['prssr'] = dft_test['prssr'].str.extract(r'(\d+\.\d+|\d+)').astype(float)
dft_test['hum'] = dft_test['hum'].str.extract(r'(\d+\.\d+|\d+)').astype(float)
dft_test['wind_spd'] = dft_test['wind_spd'].str.extract(r'(\d+\.\d+|\d+)').astype(float)
dft_test['wind_deg'] = dft_test['wind_deg'].str.extract(r'(\d+\.\d+|\d+)').astype(float)
dft_test["clouds"] = dft_test["clouds"].str.extract(r'(\d+\.\d+|\d+)').astype(float)

dft_test["rain_3h"] = 0
dft_test["snow_1h"] = 0
dft_test["snow_3h"] = 0

In [ ]:
dft_test

In [ ]:
dft_test.isna().sum()

In [ ]:
dft_test.drop(columns=columns_to_drop, inplace=True)

In [ ]:
dft_test.isna().sum()

In [ ]:
mean_train_dft = dft['d_point'].mean()
dft_test['d_point'].fillna(mean_train_dft, inplace=True)

In [ ]:
#columns_to_drop = ['time-zone']
#dft_test.drop(columns=columns_to_drop, inplace=True)

In [ ]:
dft_test

In [ ]:
result = dft_test[['datetime_iso']]
dft_test = dft_test.drop(['time-zone', 'datetime_iso'], axis=1)

In [ ]:
result

In [ ]:
y_pred_new = model_lr.predict(dft_test)
result['rain_1h'] = y_pred_new

In [ ]:
result

In [ ]:
result.to_csv('submission linier.csv', index=False)